In [2]:
import pandas as pd

In [1]:
from minio import Minio
from minio.error import S3Error

In [2]:
minio_client = Minio(
                        endpoint='localhost:9000',
                        access_key='User',
                        secret_key='p4ssw0rd',
                        secure=False)

# Nombre del bucket en MinIO
bucket_name = 'bucketbiblos'

In [3]:
try:
    objetos = minio_client.list_objects(bucket_name, recursive=True)
    # Filtrar los objetos que tienen extensión .json
    archivos_json = [obj.object_name for obj in objetos if obj.object_name.endswith('.json')]
    # Imprimir la lista de archivos JSON
    print("Archivos JSON en el bucket:")
    for archivo in archivos_json:
        print(archivo)
except S3Error as err:
    print(err)

Archivos JSON en el bucket:
url1.json
url2.json
url3.json


In [4]:
import json

datos_json ={}
for archivo in archivos_json:
    try:
        print(archivo)
        data = minio_client.get_object(bucket_name, archivo)
    # Leer los datos del archivo
        json_data = data.read().decode('utf-8')
    # Trabajar con los datos (por ejemplo, imprimirlos)
        print(json.loads( json_data))
        datos_json[archivo] =json.loads( json_data)
    except S3Error as err:
        print(err)

url1.json
{'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:3857'}}, 'features': [{'type': 'Feature', 'id': 13, 'geometry': {'type': 'Point', 'coordinates': [-8244653.3674, 530074.8121000007]}, 'properties': {'OBJECTID': 13, 'POINT_X': -74.06298132, 'POINT_Y': 4.75627098, 'LECCODIGO': 15, 'LECNOMBRE': 'Centro Cultural y Biblioteca Pública Julio Mario Santo Domingo', 'LECDIRECCI': 'Avenida calle 170 No. 67-51\xa0', 'LECTELEFON': '57 1 - 5803080', 'LECEMAIL': 'direccion_jm@biblored.gov.co', 'LECPAGWEB': 'http://www.biblored.gov.co/index.php/bibliotecas', 'LECCODLOC': '11', 'LECNOMLOC': 'Suba', 'LECCODUPZ': 'UPZ18', 'LECNOMUPZ': 'Britalia', 'LECCODSEC': '009134', 'LECNOMSEC': 'San José V Sector', 'LECESTADO': 'Activo', 'LECNODO': 'Nodo Julio Mario Santo Domingo', 'LECCLASIF': 'BibloRed', 'LECANIO': '2016', 'LECCONTACTO': 'N.A'}}, {'type': 'Feature', 'id': 14, 'geometry': {'type': 'Point', 'coordinates': [-8247488.8803, 518995.93659999967]}, 'properties': {

In [5]:
datos_json['url1.json'].keys()

dict_keys(['type', 'crs', 'features'])

In [6]:

for biblo in datos_json['url1.json']['features']:
    biblo=biblo['properties']
    print(biblo['OBJECTID']," ", biblo['LECNOMBRE'])

13   Centro Cultural y Biblioteca Pública Julio Mario Santo Domingo
14   Biblioteca Pública Virgilio Barco
15   Biblioteca Pública Arborizadora Alta
16   Biblioteca Pública El Tunal Gabriel García Márquez
17   Biblioteca Pública Usaquén Servita
18   Biblioteca Pública Venecia Pablo de Tarso
19   Biblioteca Pública La Giralda
20   Biblioteca Público Escolar La Marichuela
21   Biblioteca Publico Escolar de Sumapaz
22   Biblioteca Público Escolar Pasquilla
23   Biblioteca Pública El Parque
24   Biblioteca Instituto Geográfico Agustín Codazzi
25   Biblioteca Luis Carlos Galán Sarmiento - Biblioteca del Congreso
26   Biblioteca Luis Ángel Arango
27   Biblioteca Nacional de Colombia
28   Biblioteca especializada en Historia Política - FUGA
29   Biblioteca Cárcel Distrital
30   Biblioteca Universitaria de los Andes
31   Biblioteca Universitaria Jorge Tadeo Lozano
32   Biblioteca Universitaria de la Nacional
1   Biblioteca Pública del Deporte
2   Biblioteca Pública Carlos E. Restrepo
3   Bibli

In [1]:
import psycopg2

# Parámetros de conexión
dbname = 'postgres'
user = 'postgres'
password = '123postgis'
host = 'localhost'  # Cambia esto si tu base de datos está en otro host
port = '5432'  # Puerto pre determinado de PostgreSQL

# Crear una conexión
try:
    conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port)
    print("connection exitosa")
except Exception as e:
    print("Error", e)




connection exitosa


In [2]:


# Crear un cursor para ejecutar consultas
cur = conn.cursor()

# Ejecutar una consulta
cur.execute('SELECT * FROM "user"')

# Obtener los resultados
rows = cur.fetchall()
for row in rows:
    print(row)

# Cerrar el cursor y la conexión
cur.close()
conn.close()


(1, 'Usuario 1')
(2, 'Usuario 2')
(3, 'Usuario 3')
(4, 'Usuario 4')
(5, 'Usuario 5')


In [32]:
from src.app_biblio.services.DatabaseConnection import DatabaseConnection

In [33]:
def get_bibliotecas():
        description = []
        db_connection = DatabaseConnection()
        db_connection.connect()
        query = """
            SELECT descripcion FROM libros
        """
        cursor = db_connection.execute(query)
        rows = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        description = [dict(zip(columns, row)) for row in rows]
        cursor.close()
        db_connection.disconnect()
        return description


In [34]:
# Función para limpiar el texto de caracteres especiales
import re

def limpiar_texto(texto):
    # Elimina caracteres especiales y deja solo el texto
    texto_limpio = re.sub(r'[^\w\s.,]', '', texto)
    # Codifica el texto limpio a ASCII, ignorando los caracteres que no se pueden codificar
    texto_limpio = texto_limpio.encode('ascii', errors='ignore').decode()
    return texto_limpio.strip()

In [ ]:
get_bibliotecas()

connect to database


[{'descripcion': 'Fecha tomada de la cabecera  // Hoja impresa por una sola cara  // Texto a dos columnas y con orla tipográfica  // Grabado en cabecera  // '},
 {'descripcion': 'Texto a tres columnas y con orla tipográfica  // Hoja impresa por una sola cara  // Grabado xilográfico en cabecera  // '},
 {'descripcion': 'Fecha de impresión deducida de los años de trabajo del impresor  // Texto a tres columnas y con orla tipográfica  // Grabado xilográfico en cabecera  // '},
 {'descripcion': 'Texto a dos columnas y con orla tipográfica  // Hoja impresa por una sola cara  // Grabado xilográfico en cabecera  // '},
 {'descripcion': 'Fecha de impresión deducida de los años de trabajo del impresor  // Texto a tres columnas y con orla tipográfica  // Hoja impresa por una sola cara  // Grabado en cabecera  // '},
 {'descripcion': 'Fecha de impresión deducida de los años de trabajo del impresor  // Texto a tres columnas y con orla tipográfica  // Grabado en cabecera  // '},
 {'descripcion': 'Fe

In [35]:
from fpdf import FPDF

# Definir la función get_bibliotecas() y limpiar_texto() según sea necesario

pdf = FPDF()
pdf.add_page()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.set_font("Arial", size=12)

descripciones = get_bibliotecas()  # Suponiendo que get_bibliotecas() obtiene las descripciones de alguna manera

for descripcion in descripciones:
    try:
        descripcion_limpia = limpiar_texto(descripcion['descripcion'])
        pdf.multi_cell(0, 10, descripcion_limpia + "\n")
    except Exception as e:
        print("Se produjo un error al procesar una descripción:", e)

pdf.output("descripciones.pdf")


connect to database


''

In [54]:
# Function to convert PDF to text
import PyPDF2
def pdf_to_text(file_path):
    pdf_file = open(file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page_num in range( len(pdf_reader.pages)):
        text += pdf_reader.pages[page_num].extract_text()
    pdf_file.close()
    return text

In [60]:

import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [61]:

# Initialize text splitter and embeddings
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
#embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
#embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = HuggingFaceEmbeddings(model_name="hiiamsid/sentence_similarity_spanish_es")
client = chromadb.PersistentClient(path="./db")
try:
    collection = client.create_collection(name="my_collection")
except :
    collection = client.get_or_create_collection(name="my_collection")

In [62]:
# Process each PDF in the ./input directory
import os

for filename in os.listdir('./input'):
    
    if filename.endswith('.pdf'):
        print(str(filename))
        # Convert PDF to text
        text = pdf_to_text(os.path.join('./input', filename))

        # Split text into chunks
        chunks = text_splitter.split_text(text)

        # Convert chunks to vector representations and store in Chroma DB
        documents_list = []
        embeddings_list = []
        ids_list = []

        for i, chunk in enumerate(chunks):
            vector = embeddings.embed_query(chunk)

            documents_list.append(chunk)
            embeddings_list.append(vector)
            ids_list.append(f"{filename}_{i}")


        collection.add(
            embeddings=embeddings_list,
            documents=documents_list,
            ids=ids_list
        )

descripciones.pdf


KeyboardInterrupt: 

In [ ]:

query = input("Enter your query: ")

# Convert query to vector representation
query_vector = embeddings.embed_query(query)

# Query Chroma DB with the vector representation
results = collection.query(query_embeddings=query_vector, n_results=3 , include=["documents"])

# Print results
for result in results["documents"]:
    for i in result:
        print(f"\n ######## Chunk id {i} \n")
        print(i)